In [1]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

In [2]:
from crewai import Agent, Task, Crew
from crewai_tools import ScrapeWebsiteTool
from crewai.memory import LongTermMemory
from crewai.memory.storage.ltm_sqlite_storage import LTMSQLiteStorage

/home/patrick/.pyenv/versions/3.11.11/lib/python3.11/site-packages/pydantic/_internal/_config.py:323: PydanticDeprecatedSince20: Support for class-based `config` is deprecated, use ConfigDict instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.11/migration/
  warnings.warn(DEPRECATION_MESSAGE, DeprecationWarning)
/home/patrick/.pyenv/versions/3.11.11/lib/python3.11/site-packages/pydantic/_internal/_config.py:323: PydanticDeprecatedSince20: Support for class-based `config` is deprecated, use ConfigDict instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.11/migration/
  warnings.warn(DEPRECATION_MESSAGE, DeprecationWarning)
/home/patrick/.pyenv/versions/3.11.11/lib/python3.11/site-packages/crewai_tools/tools/scrapegraph_scrape_tool/scrapegraph_scrape_tool.py:34: PydanticDeprecatedSince20: Pydantic V1 style `@validator` validators are deprecated

In [3]:
import os
from utils import get_openai_api_key

openai_api_key = get_openai_api_key()
os.environ["OPENAI_MODEL_NAME"] = 'gpt-4o-mini'

In [4]:
support_agent = Agent(
    role="Senior Support Representative",
	goal="Be the most friendly and helpful "
        "support representative in your team",
	backstory=(
		"You work at crewAI (https://crewai.com) and "
        " are now working on providing "
		"support to {customer}, a super important customer "
        " for your company."
		"You need to make sure that you provide the best support!"
		"Make sure to provide full complete answers, "
        " and make no assumptions."
	),
	allow_delegation=False,
	verbose=True
)

In [5]:
support_quality_assurance_agent = Agent(
	role="Support Quality Assurance Specialist",
	goal="Get recognition for providing the "
    "best support quality assurance in your team",
	backstory=(
		"You work at crewAI (https://crewai.com) and "
        "are now working with your team "
		"on a request from {customer} ensuring that "
        "the support representative is "
		"providing the best support possible.\n"
		"You need to make sure that the support representative "
        "is providing full"
		"complete answers, and make no assumptions."
	),
	verbose=True
)

In [6]:
docs_scrape_tool = ScrapeWebsiteTool(
    website_url="https://docs.crewai.com/concepts/memory"
)

In [7]:
inquiry_resolution = Task(
    description=(
        "{customer} just reached out with a super important ask:\n"
	    "{inquiry}\n\n"
        "{person} from {customer} is the one that reached out. "
		"Make sure to use everything you know "
        "to provide the best support possible."
		"You must strive to provide a complete "
        "and accurate response to the customer's inquiry."
    ),
    expected_output=(
	    "A detailed, informative response to the "
        "customer's inquiry that addresses "
        "all aspects of their question.\n"
        "The response should include references "
        "to everything you used to find the answer, "
        "including external data or solutions. "
        "Ensure the answer is complete, "
		"leaving no questions unanswered, and maintain a helpful and friendly "
		"tone throughout."
    ),
	tools=[docs_scrape_tool],
    agent=support_agent,
)

In [8]:
quality_assurance_review = Task(
    description=(
        "Review the response drafted by the Senior Support Representative for {customer}'s inquiry. "
        "Ensure that the answer is comprehensive, accurate, and adheres to the "
		"high-quality standards expected for customer support.\n"
        "Verify that all parts of the customer's inquiry "
        "have been addressed "
		"thoroughly, with a helpful and friendly tone.\n"
        "Check for references and sources used to "
        " find the information, "
		"ensuring the response is well-supported and "
        "leaves no questions unanswered."
    ),
    expected_output=(
        "A final, detailed, and informative response "
        "ready to be sent to the customer.\n"
        "This response should fully address the "
        "customer's inquiry, incorporating all "
		"relevant feedback and improvements.\n"
		"Don't be too formal, we are a chill and cool company "
	    "but maintain a professional and friendly tone throughout."
    ),
    agent=support_quality_assurance_agent,
)


In [9]:
crew = Crew(
  agents=[support_agent, support_quality_assurance_agent],
  tasks=[inquiry_resolution, quality_assurance_review],
  verbose=True,
  memory=True,
  long_term_memory=LongTermMemory(
        storage=LTMSQLiteStorage(
            db_path="03_muilti_agent_customer_support.db"
        )
    ),
)

In [10]:
inputs = {
    "customer": "DeepLearningAI",
    "person": "Andrew Ng",
    "inquiry": "I need help with setting up a Crew "
               "and kicking it off, specifically "
               "how can I add memory (short, long-term and entity) to my crew agents running locally or in my own server? "
               "Can you provide guidance?"
}

result = crew.kickoff(inputs=inputs)

╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: 30b557a6-7a8e-4898-9c6f-b28fb8f88706                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

🚀 Crew: crew
└── 📋 Task: 337fb178-4ed8-4513-91e6-1a74e3178b15
       Status: Executing Task...

🚀 Crew: crew
└── 📋 Task: 337fb178-4ed8-4513-91e6-1a74e3178b15
       Status: Executing Task...
    └── 🤖 Agent: Senior Support Representative
            Status: In Progress

# Agent: Senior Support Representative
## Task: DeepLearningAI just reached out with a super important ask:
I need help with setting up a Crew and kicking it off, specifically how can I add memory (short, long-term and entity) to my crew agents running locally or in my own server? Can you provide guidance?

Andrew Ng from DeepLearningAI is the one that reached out. Make sure to use everything you know to provide the best support possible.You must strive to provide a complete and accurate response to the customer's inquiry.


🚀 Crew: crew
└── 📋 Task: 337fb178-4ed8-4513-91e6-1a74e3178b15
       Status: Executing Task...
    └── 🤖 Agent: Senior Support Representative
            Status: In Progress
        └── 🧠 Thinking...

...

/home/patrick/.pyenv/versions/3.11.11/lib/python3.11/site-packages/chromadb/types.py:144: PydanticDeprecatedSince211: Accessing the 'model_fields' attribute on the instance is deprecated. Instead, you should access this attribute from the model class. Deprecated in Pydantic V2.11 to be removed in V3.0.
  return self.model_fields  # pydantic 2.x
/home/patrick/.pyenv/versions/3.11.11/lib/python3.11/site-packages/chromadb/types.py:144: PydanticDeprecatedSince211: Accessing the 'model_fields' attribute on the instance is deprecated. Instead, you should access this attribute from the model class. Deprecated in Pydantic V2.11 to be removed in V3.0.
  return self.model_fields  # pydantic 2.x


🚀 Crew: crew
├── 📋 Task: 337fb178-4ed8-4513-91e6-1a74e3178b15
│      Assigned to: Senior Support Representative
│      Status: ✅ Completed
│   └── 🤖 Agent: Senior Support Representative
│           Status: ✅ Completed
│       └── 🔧 Used Read website content (1)
└── 📋 Task: a778bb11-54e4-4cb2-bfd6-be4b7c872239
       Status: Executing Task...
    └── 🤖 Agent: Support Quality Assurance Specialist
            Status: ✅ Completed

🚀 Crew: crew
├── 📋 Task: 337fb178-4ed8-4513-91e6-1a74e3178b15
│      Assigned to: Senior Support Representative
│      Status: ✅ Completed
│   └── 🤖 Agent: Senior Support Representative
│           Status: ✅ Completed
│       └── 🔧 Used Read website content (1)
└── 📋 Task: a778bb11-54e4-4cb2-bfd6-be4b7c872239
       Assigned to: Support Quality Assurance Specialist
       Status: ✅ Completed
    └── 🤖 Agent: Support Quality Assurance Specialist
            Status: ✅ Completed

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: a778bb11-54e4-4cb2-bfd6-be4b7c872239                                                                     │
│  Agent: Support Quality Assurance Specialist                                                                    │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Crew Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Completed                                                                                       │
│  Name: crew                                                                                                     │
│  ID: 30b557a6-7a8e-4898-9c6f-b28fb8f88706                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

In [11]:
from IPython.display import Markdown
Markdown(result.raw)

Hello Andrew!

Thank you for reaching out to us at DeepLearningAI. We’re excited to assist you in setting up a Crew with memory capabilities on your local server. Below is a detailed guide that covers all aspects of adding memory to your Crew agents effectively:

### Memory Overview
CrewAI offers a comprehensive memory system that significantly enhances your AI agents’ capabilities. You have several types of memory to choose from:

1. **Short-Term Memory**: This allows agents to remember recent interactions during ongoing tasks, making them more responsive and relevant in conversations.
2. **Long-Term Memory**: It helps retain important insights and learnings from past interactions, enabling your agents to develop a richer knowledge base over time.
3. **Entity Memory**: This captures detailed information about specific entities (e.g., people, places, or concepts), which aids in creating better context and understanding relationships.
4. **Contextual Memory**: This combines Short-Term, Long-Term, and Entity memory to keep track of conversations' context, ensuring coherent and relevant agent responses throughout interactions.

### Setting Up the Crew with Memory
To enable and customize memory components for your Crew, please follow these steps:

1. **Activate the Memory System**: The memory system is disabled by default. To enable it, set `memory=True` in your crew's configuration.
2. **Customize Your Memory Settings**: Adjust various aspects, including specifying embedding models for memory storage based on your needs.

### Example Configuration
Here’s a practical example of how to configure your Crew with the desired memory capabilities:

```python
from crewai import Crew, Process
from crewai.memory import LongTermMemory, ShortTermMemory, EntityMemory
from crewai.memory.storage import RAGStorage, LTMSQLiteStorage

my_crew = Crew(
    agents=[...],  # Define your agents here
    tasks=[...],   # Define your tasks here
    process=Process.sequential,
    memory=True,
    
    # Configuring Long-Term Memory
    long_term_memory=LongTermMemory(
        storage=LTMSQLiteStorage(
            db_path="/my_crew1/long_term_memory_storage.db"
        )
    ),

    # Configuring Short-Term Memory
    short_term_memory=ShortTermMemory(
        storage=RAGStorage(
            embedder_config={
                "provider": "openai",
                "config": {
                    "model": 'text-embedding-3-small'  # Choose the appropriate embedding model
                }
            },
            type="short_term",
            path="/my_crew1/"
        )
    ),

    # Configuring Entity Memory
    entity_memory=EntityMemory(
        storage=RAGStorage(
            embedder_config={
                "provider": "openai",
                "config": {
                    "model": 'text-embedding-3-small'  # Choose the appropriate embedding model
                }
            },
            type="short_term",
            path="/my_crew1/"
        )
    ),
    
    verbose=True,
)
```

### Important Notes
- **Storage Paths**: Make sure to manage your storage paths carefully and avoid hardcoding sensitive information to protect against data breaches. This is crucial for maintaining security.
- **Resetting Memory**: If you ever need to clear any memory types, you can use the `reset_memories` method available within the Crew object for easy management of your agents' memory.
- **Integration of Advanced Memory Features**: Consider utilizing the Mem0 API to incorporate user-specific memory capabilities for a more personalized interaction experience.

By following these guidelines, you should be on the right track to launching your Crew with the desired memory functionalities! If you have any further questions or need more assistance, please don’t hesitate to reach out. We’re here to support you every step of the way!

Best regards,  
[Your Name]  
CrewAI Support Team